## MINST 手写字体识别 - LeNet: CNN, conv + pooling + CPU, Yann LeCun, 1998 
## ImageNet - AlexNet: ReLU + GPU, Hinton and Alex Krizhevsky, 2012
## ImageNet - VGGNet: 3 x 3 conv, Google Deep Mind, 2014
## ImageNet - ResNet: 解决梯度消失现象, 2015

In [ ]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms

In [ ]:
torch.cuda.set_device(2)

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [ ]:
trainset = datasets.MNIST('data', train = True, download = True, transform = transform)
testset = datasets.MNIST('data', train = False, download = True, transform = transform)

In [ ]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv_1 = nn.Conv2d(1, 6, (5, 5)) # 输入 1 张灰度图, 输出 6 张特征图, 5 * 5 的卷积核
        self.conv_3 = nn.Conv2d(6, 16, (5, 5))
        self.linear_1 = nn.Linear(16*4*4, 120)
        self.linear_2 = nn.Linear(120, 84)
        self.linear_3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv_1(x)), 2) # 2 * 2 池化核
        x = F.max_pool2d(F.relu(self.conv_3(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.linear_1(x))
        x = F.relu(self.linear_2(x))
        x = self.linear_3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features = num_features * s
        return num_features

In [ ]:
CUDA = torch.cuda.is_available()
if CUDA:
    lenet = LeNet().cuda()
else:
    lenet = LeNet()

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
optimizer = optim.SGD(lenet.parameters(), lr = 0.001, momentum = 0.9)

In [ ]:
# 一次性加载的数据量为 4, 遍历不同批次的数据时打乱顺序, 使用 4 个子进程加载数据
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 4, shuffle = True, num_workers = 4) 

In [ ]:
def train(model, criterion, optimizer, epochs = 1):
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0): # 第 0 项开始对 trainloader 数据进行枚举, 返回序号 i, 数据和标签 data
            inputs, labels = data
            if CUDA:
                inputs, labels = inputs.cuda(), labels.cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss = running_loss + loss.item()
            if i % 1000 == 999:
                print('[Epoch: %d, Batch: %5d] Loss: %.3f' % (epoch + 1, i + 1, running_loss / 1000))
                running_loss = 0.0
            
    print('Finished training.')

In [ ]:
train(lenet, criterion, optimizer, epochs = 2) # 完全遍历训练数据 2 次, 每 1000 次打印一次 loss 均值